In [8]:
import os
from dotenv import load_dotenv

### Loading the environment Variables

In [9]:
load_dotenv()

True

### Obtaining the LLM Model

In [10]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.7
)

In [11]:
t=llm.invoke("who are you ?")
t.content

'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'

### Web Based Scraping 

In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://www.google.com/about/careers/applications/jobs/results/116221099686929094-account-executive-midmarket-sales-google-customer-solutions")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [13]:
page_data = loader.load().pop().page_content
page_data

"Account Executive, Mid-Market Sales, Google Customer Solutions — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results1,840  jobs matchedAccount Executive, Mid-Market Sales, Google Customer SolutionsMumbai, Maharashtra, India; Gurgaon, Haryana, India; +2 more; +1 moreSoftware Engineer III, Infrastructure, CoreBengaluru, Karnataka, IndiaSenior Software Engineer, AI/ML GenAI, Google Cloud AISunnyvale, CA, USA; Kirkland, WA, USASenior Program Manager, Supply Chain Operations, Hardware FulfillmentSunnyvale, CA, USASoftware Engineer, Silicon Engineering ProductivityBengaluru, Karnataka, IndiaData Center Security ManagerSkien, NorwayTechnical Program Manager II, Infrastructure Resource Management, YouTubeMountain Vie

### Prompt Template for LLM Model

In [14]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser


template =PromptTemplate.from_template ("""
        ### Scraped Text From Website :
        {page_data}
        ## Instruction  
        Based on the data given above is the content of career's page of a website.
        You are requiremt to extract the job posting and return them in JSON fromat with no PREAMBLE.
        These should be the keys: 'role', 'expereince' ,'skills' and 'description' .
        ONLY return VALID JSON.                              
        """)

parser = JsonOutputParser()

chain = template | llm | parser
response=chain.invoke(input={'page_data':page_data})
response

{'role': 'Account Executive, Mid-Market Sales, Google Customer Solutions',
 'experience': '5 years of experience in sales, advertising, or marketing',
 'skills': ['Knowledge of traditional and online marketing strategy and tactics',
  'Knowledge of the local market with excellent analytical skills',
  'Ability to think strategically about complex issues and develop recommendations and action plans',
  'Demonstrated ability to lead and manage multiple projects with a broad range of internal and external constituents simultaneously'],
 'description': 'As an Account Executive, you will work with your respective set of advertisers to shape their business growth and strengthen long-term relationships. You will drive scalable product adoption and business growth. In this role, you will anticipate how decisions are made at a C-Level, you will explore and uncover the business needs of customers, and understand how our range of product offerings can grow their business.'}

In [15]:
import pandas as pd

In [16]:
df = pd.read_csv("my_portfolio.csv")

In [17]:
df.head(3)

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio


### Creating a Vector Database using ChromaDB

In [18]:
import chromadb
import uuid
chroma_client = chromadb.PersistentClient()

collection = chroma_client.get_or_create_collection(name="TechStack")

if not collection.count():
    for _,row in df.iterrows():
        document_id = str(uuid.uuid4())

        collection.add(
            documents=[row['Techstack']], 
            metadatas=[{'link': row['Links']}],  
            ids=[document_id]  
        )

In [19]:
links = collection.query(
    query_texts=['Experience in Python','Expereicne in React'],
    n_results=2
)['metadatas']
type(links)

list

In [20]:
links

[[{'link': 'https://example.com/ml-python-portfolio'},
  {'link': 'https://example.com/python-portfolio'}],
 [{'link': 'https://example.com/react-portfolio'},
  {'link': 'https://example.com/react-native-portfolio'}]]

### Integrating the Initial LLM response With the DataBase


In [25]:
print(type(response),end=" ")
output=response


<class 'dict'> 

In [26]:
output

{'role': 'Account Executive, Mid-Market Sales, Google Customer Solutions',
 'experience': '5 years of experience in sales, advertising, or marketing',
 'skills': ['Knowledge of traditional and online marketing strategy and tactics',
  'Knowledge of the local market with excellent analytical skills',
  'Ability to think strategically about complex issues and develop recommendations and action plans',
  'Demonstrated ability to lead and manage multiple projects with a broad range of internal and external constituents simultaneously'],
 'description': 'As an Account Executive, you will work with your respective set of advertisers to shape their business growth and strengthen long-term relationships. You will drive scalable product adoption and business growth. In this role, you will anticipate how decisions are made at a C-Level, you will explore and uncover the business needs of customers, and understand how our range of product offerings can grow their business.'}

In [27]:
from langchain.document_loaders import PyPDFLoader

pdf_path = "Atir_Shakhrelia_SDE.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()
pdf_text = " ".join([document.page_content for document in documents])
pdf_text

'ATIR SHAKHRELIA\nBoston, MA | +1 (857) 605-9479 | shakhrelia.a@northeastern.edu | LinkedIn | GitHub\nEDUCATION\nNortheastern University - Khoury College of Computer Sciences Boston, USA\nMaster of Science, Computer Science, GPA: 4.0 May 2026\n◦ Relevant Coursework: Programming Design Paradigms, Data Management and Processing, Algorithms,\nMobile Application Development\nNirma University - Institute of Technology Ahmedabad, India\nBachelor of Technology, Computer Science and Engineering, GPA: 4.0 Jun 2024\n◦ Relevant Coursework: Data Structures and Algorithms, Object Oriented Programming, Operating Systems,\nDatabases, Machine Learning, Deep Learning, Big Data Technologies, Data Mining, Blockchain Technology\nEXPERIENCE\nJohnson Controls-Hitachi Ahmedabad, India\nResearch and Development Intern Jan 2024 - Jun 2024\n* Developed a robust Bluetooth communication infrastructure usingSilicon Labs EK4108-A.\n* Implemented a nested hashing algorithm, achieving a 33% increase in non-line-of-si

In [29]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### RESUME CONTENT
        {resume_text}

        ### INSTRUCTION:
        You are an expert cold-email generation. On the basis of the job description and the resume  provided above generate a cold-email for the 
        job descriton.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm

res = chain_email.invoke({"job_description":output['description'],"link_list":links,"resume_text":pdf_text}).content

print(res)

Subject: Driving Business Growth through Innovative Solutions

Dear Hiring Manager,

I came across the Account Executive role at your esteemed organization, and I am excited to introduce myself as a strong candidate who can drive scalable product adoption and business growth. With a solid educational background in Computer Science and a proven track record of delivering high-impact projects, I am confident in my ability to shape business growth and strengthen long-term relationships with your advertisers.

As a highly motivated and results-driven individual, I have developed a unique blend of technical and business skills, which enables me to understand the needs of customers and provide tailored solutions. My experience in research and development, web development, and academic projects has equipped me with a strong foundation in programming languages, data management, and machine learning. I am excited about the opportunity to leverage my skills to drive business growth and explore n